In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm_notebook
from tqdm import tqdm

In [2]:
pd.set_option('max_colwidth', 500)
# pd.set_option('display.width', 120)
# pd.set_option('display.max_rows', 36)

In [3]:
# считываем данные

train_labels = pd.read_csv("../data/external/train_labels.csv", sep=';')
train_data = pd.read_csv("../data/external/train_data.csv", sep=';')
test_data = pd.read_csv("../data/external/test_data.csv", sep=';')


recommendations_svd = pd.read_csv("../data/processed/recommendations_svd.csv", sep=';')
recommendations_doc2vec = pd.read_csv("../data/processed/recommendations_doc2vec.csv", sep=';')

C:\Users\FED\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\FED\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# заполняем пропуски, объединяем окпд2 и дополнительный код, объединяем описания

train_data = train_data.fillna('None')
test_data = test_data.fillna('None')

train_data['okpd2_or_additional_code'] = train_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)
test_data['okpd2_or_additional_code'] = test_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)

train_data['text_description_tender'] = train_data['purchase_name'] + " " + train_data['lot_name'] + " " + train_data['okpd2_names'] + " " + train_data['additional_code_names'] + " " + train_data['item_descriptions']
test_data['text_description_tender'] = test_data['purchase_name'] + " " + test_data['lot_name'] + " " + test_data['okpd2_names'] + " " + test_data['additional_code_names'] + " " + test_data['item_descriptions']

In [5]:
def show_lots(user_id):
    '''
    
    функция для показа истории и рекомендаций для участника
    
    '''
    history_ = train_labels[train_labels['participant_inn_kpp_anon'] == user_id].merge(train_data, how='left', left_on='pn_lot_anon', right_on='pn_lot_anon')[['fz_x', 'region_code', 'okpd2_or_additional_code', 'text_description_tender']]
    recomended_lots_1 = recommendations_svd[recommendations_svd.inn_kpp == user_id].merge(test_data,  how='left', left_on='actual_recommended_pn_lot', right_on='pn_lot_anon')[['fz', 'region_code', 'okpd2_or_additional_code', 'text_description_tender']]
    recomended_lots_2 = recommendations_doc2vec[recommendations_doc2vec.inn_kpp == user_id].merge(test_data,  how='left', left_on='actual_recommended_pn_lot', right_on='pn_lot_anon')[['fz', 'region_code', 'okpd2_or_additional_code', 'text_description_tender']]
    history_.to_csv("../data/processed/history.csv", index=False)
    recomended_lots_1.to_csv("../data/processed/rec_1.csv", index=False)
    recomended_lots_2.to_csv("../data/processed/rec_2.csv", index=False)

    return 'Done'

In [24]:
# рекомендации для случайного участника

show_lots(recommendations_svd.inn_kpp.sample().values[0])

'Done'

In [25]:
pd.read_csv("../data/processed/history.csv")

,fz_x,region_code,okpd2_or_additional_code,text_description_tender
0,44fz,91,85.4,Оказание услуг по проведению профессиональной переподготовки специалистов по охране труда исполнительных органов государственной власти Республики Крым и органов местного самоуправления муниципальных образований в Республике Крым None Услуги по профессиональному обучению прочие None Оказание услуг по проведению профессиональной переподготовки специалистов по охране труда исполнительных органов государственной власти Республики Крым и органов местного самоуправления муниципальных образований ...
1,44fz,91,85.4,"Оказание образовательных услуг - подготовка специалистов, ответственных за организацию работ по техническому обслуживанию и ремонту лифтов None Услуги в области дополнительного образования прочие, не включенные в другие группировки None Оказание образовательных услуг - подготовка специалистов, ответственных за организацию работ по техническому обслуживанию и ремонту лифтов Оказание образовательных услуг - подготовка специалистов, ответственных за организацию работ по техническому обслуживани..."
2,44fz,91,85.4,Оплата курсов по повышению квалификации по программе: Противодействие коррупции None Услуги по профессиональному обучению прочие None Оплата курсов по повышению квалификации по программе: Противодействие коррупции В соответствии с условиями государственного контракта
3,44fz,91,85.4,"Оказание услуг по проведению профессиональной переподготовки специалистов по охране труда предприятий, организаций и учреждений бюджетной сферы Республики Крым в области охраны труда None Услуги по профессиональному обучению прочие None Оказание услуг по проведению профессиональной переподготовки специалистов по охране труда предприятий, организаций и учреждений бюджетной сферы Республики Крым в области охраны труда"
4,44fz,91,85.4,Образовательные услуги по повышению квалификации государственных гражданских служащих Республики Крым по программе: Охрана труда работников организаций None Услуги по профессиональному обучению прочие None Образовательные услуги по повышению квалификации государственных гражданских служащих Республики Крым по программе: Охрана труда работников организаций
...,...,...,...,...
60,44fz,91,85.3,Обучение по курсу «Безопасные методы и приемы выполнения работ на высоте для работников 3 группы» None Услуги по профессиональному обучению None Обучение по курсу «Безопасные методы и приемы выполнения работ на высоте для работников 3 группы»
61,44fz,91,85.3,Пожарно-технический минимум для руководителей и специалистов. Пожарно-технический минимум для электрогазосварщиков. Обучение по программе «Правила по охране труда на автомобильном транспорте» None Услуги по профессиональному обучению None Пожарно-технический минимум для руководителей и специалистов. Пожарно-технический минимум для электрогазосварщиков. Обучение по программе «Правила по охране труда на автомобильном транспорте»
62,44fz,91,85.3,Повышение квалификации по программе: Обеспечение экологической безопасности руководителями и специалистами общехозяйственных систем управления Повышение квалификации по программе: Обеспечение экологической безопасности при работах в области обращения с опасными отходами None Услуги по профессиональному обучению None Повышение квалификации по программе: Обеспечение экологической безопасности руководителями и специалистами общехозяйственных систем управления Повышение квалификации по програ...
63,44fz,91,85.4,Профессиональная переподготовка специалистов по безопасности дорожного движения на автомобильном транспорте None Услуги по профессиональному обучению прочие None Профессиональная переподготовка специалистов по безопасности дорожного движения на автомобильном транспорте


In [26]:
pd.read_csv("../data/processed/rec_1.csv")

,fz,region_code,okpd2_or_additional_code,text_description_tender
0,44fz,91,85.4,Оказание услуг по обучению специалистов инженерных подразделений None Услуги по дополнительному профессиональному образованию прочие None Оказание услуг по обучению специалистов инженерных подразделений
1,44fz,91,85.4,Услуги по повышению квалификации Экономический анализ финансово-хозяйственной деятельности организации None Услуги по профессиональному обучению прочие None Услуги по повышению квалификации Экономический анализ финансово-хозяйственной деятельности организации
2,44fz,91,85.4,"Обучение специалистов по охране труда, обучение работников, выполняющих работы, связанные с хранением, транспортировкой и обслуживанием кислородных баллонов в медицинских учреждениях, электромонтер по обслуживанию электрических установок - допуск к работе электроустановок напряжения до 1000в и лифтеров» None Услуги по дополнительному профессиональному образованию прочие None Обучение специалистов по охране труда, обучение работников, выполняющих работы, связанные с хранением, транспортировко..."
3,44fz,91,85.4,"Оказание услуг по обучению сотрудников в рамках Федерального закона от 05.04.2013 № 44-ФЗ О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд. None Услуги по профессиональному обучению прочие None оказание услуг по обучению сотрудников в рамках Федерльного закона от 05.04.2013 № 44-ФЗ О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд"
4,44fz,92,85.4,"На оказание образовательных услуг None Услуги в области дополнительного образования прочие, не включенные в другие группировки None Услуги в области дополнительного образования прочие"
5,44fz,92,85.4,На оказание услуг по обучению охране труда и проверке знаний требований охраны труда руководителей и специалистов None Услуги по профессиональному обучению прочие None Услуги по профессиональному обучению прочие
6,44fz,91,85.4,"Оказание услуг по проведению профессиональной подготовки по курсу «Обучение по охране труда и проверка знаний, требований охраны труда руководителей и специалистов» None Услуги по профессиональному обучению прочие None Оказание услуг по проведению профессиональной подготовки по курсу «Обучение по охране труда и проверка знаний, требований охраны труда руководителей и специалистов»"
7,44fz,91,85.4,"Услуги по обучению сотрудников МБУ Город None Услуги по профессиональному обучению прочие None Обучение по охране труда руководителей и специалистов организаций || Обучение электротехнического и электротехнологического персонала по электробезопасности напряжением до 1000В с сдачей экзамена в Крымтехнадзоре || Повышение квалификации «Монтаж, наладка, ремонт, реконструкция или модернизация подъемных сооружений в процессе эксплуатации опасных производственных объектов» Б.9.33 || Повышение Кв..."
8,44fz,91,85.4,Оказание услуг по обучению должностных лиц по вопросам охраны труда и пожарно-технического минимума None Услуги по дополнительному профессиональному образованию прочие None Обучение должностных лиц по вопросам охраны труда и пожарно-технического минимума
9,44fz,92,85.4,"Оказание услуг по обучению и повышению квалификации сотрудников ГБУ ЦСПСД None Услуги в области дополнительного образования прочие, не включенные в другие группировки None Обучение по курсу «Пожарно-технический минимум для руководителей и специалистов» || Повышение квалификации по курсу «Ответственный за безопасную эксплуатацию тепловых энергоустановок»"


In [27]:
pd.read_csv("../data/processed/rec_2.csv")

,fz,region_code,okpd2_or_additional_code,text_description_tender
0,44fz,91,20.2,"Приобретение дезинфицирующих средств None Средства дезинфекционные None Дезинфицирующее средство в таблетках Должно представлять круглые таблетки средней массой не менее 0,5 г. 1,0 г. 1,5 г. 2,0 г. В качестве действующих веществ должно содержать полигексаметиленгуанидин гидрохлорид, а также другие синергетические и функциональные компоненты. Должно быть выпущено не ранее 2020 года. Срок годности в невскрытой упаковке должен быть не менее 3 лет. Должно обладать моющими средствами. Должно быть..."
1,44fz,91,85.4,"Оказание услуг по обучению сотрудников в рамках Федерального закона от 05.04.2013 № 44-ФЗ О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд. None Услуги по профессиональному обучению прочие None оказание услуг по обучению сотрудников в рамках Федерльного закона от 05.04.2013 № 44-ФЗ О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд"
2,44fz,91,85.4,Услуги по повышению квалификации Экономический анализ финансово-хозяйственной деятельности организации None Услуги по профессиональному обучению прочие None Услуги по повышению квалификации Экономический анализ финансово-хозяйственной деятельности организации
3,44fz,91,85.4,Услуги по профессиональной переподготовке Государственное и муниципальное управление None Услуги по профессиональному обучению прочие None Услуги по профессиональной переподготовке Государственное и муниципальное управление
4,44fz,91,26.4,Поставка звукового оборудования None Аппаратура для воспроизведения звука прочая || Усилители электрические звуковых частот None Акустическая система || Аппаратура звукоусиления || Микшерный усилитель || Усилитель-микшер
5,44fz,92,85.4,"Оказание услуг по повышению квалификации сотрудников ГБУ ЦСПСД None Услуги в области дополнительного образования прочие, не включенные в другие группировки None Повышение квалификации по программе «Контрактная система в сфере закупок товаров, работ и услуг для обеспечения государственных и муниципальных нужд» || Повышение квалификации по программе «Патопсихология подросткового и юношеского возраста. Психологическая диагностика расстройств личности и нарушений поведения || Повышения квалифи..."
6,44fz,91,85.4,"Обучение специалистов по охране труда, обучение работников, выполняющих работы, связанные с хранением, транспортировкой и обслуживанием кислородных баллонов в медицинских учреждениях, электромонтер по обслуживанию электрических установок - допуск к работе электроустановок напряжения до 1000в и лифтеров» None Услуги по дополнительному профессиональному образованию прочие None Обучение специалистов по охране труда, обучение работников, выполняющих работы, связанные с хранением, транспортировко..."
7,44fz,91,85.3,Услуги по профессиональному обучению (повышение квалификации и профессиональной переподготовки) медицинских работников ГБУЗ РК Черноморская ЦРБ None Услуги по профессиональному обучению None Анестезиология и реаниматология || Инфекционные болезни || Общая врачебная практика || Организация здравоохранения и общественное здоровье || Паллиативная медицинская помощь || Педиатрия || Профпатология || Стоматология || Стоматология терапевтическая || Терапия || Травматология и ортопедия ||...
8,44fz,91,32.9,"Приобретение письменных и чертежных принадлежностей, материалов и инвентаря для учебных занятий воспитанников None Карандаши простые и цветные с грифелями в твердой оболочке None Карандаш простой || Карандаши цветные 12 цв"
9,44fz,91,20.4,"Поставка моющих средств и порошка стирального None None Порошок стиральный || Средства моющие для стекол и зеркал || Средства моющие для туалетов и ванных комнат || Средства отбеливающие для стирки || Средство для мытья посуды Порошок стиральный Средство с отбеливающим эффектом--Да, Для детского белья--Нет, Назначение--Для белого белья, Тип стирки--Ручная и машинная стирка || Средства моющие для 